In [1]:
import hid
import time

# --- 配置 ---
VENDOR_ID = 0x045E   # Microsoft
PRODUCT_ID = 0x0B12  # Xbox Series X/S Controller

def format_byte(b):
    """将字节格式化为 'XX (ddd)' 的形式，方便阅读"""
    return f"{b:02X} ({b:3d})"

def print_report(raw_data):
    """以清晰的格式打印原始HID报告"""
    print("--------------------------------------------------")
    print(f"收到 {len(raw_data)} 字节报文:")
    
    formatted_bytes = [format_byte(b) for b in raw_data]
    
    # 每行打印 8 个字节，方便对齐和比较
    for i in range(0, len(formatted_bytes), 8):
        chunk = formatted_bytes[i:i+8]
        # 打印字节索引
        line_header = f"[{i:02d}-{i+len(chunk)-1:02d}] "
        print(line_header + "  ".join(chunk))
    print() # 空行以分隔

if __name__ == "__main__":
    device = None
    try:
        print(f"正在尝试打开设备: VID={VENDOR_ID:04x}, PID={PRODUCT_ID:04x}")
        device = hid.device()
        device.open(VENDOR_ID, PRODUCT_ID)
        device.set_nonblocking(True)
        print("设备连接成功!")
        print(f"  制造商: {device.get_manufacturer_string()}")
        print(f"  产品:   {device.get_product_string()}")
        print("\n--- HID 报文分析器已启动 ---")
        print("请按照以下顺序，每次只做一个动作，并保持几秒钟：")
        print("1.  保持手柄完全静止 (非常重要！)")
        print("2.  只按住 [A] 键")
        print("3.  只按住 [B] 键")
        print("4.  只按住 [X] 键")
        print("5.  只按住 [Y] 键")
        print("6.  只按住 [LB] (左肩键)")
        print("7.  只按住 [RB] (右肩键)")
        print("8.  将 [LT] (左扳机) 压到底")
        print("9.  将 [RT] (右扳机) 压到底")
        print("10. 将 [左摇杆] 向右推到底")
        print("11. 将 [左摇杆] 向上推到底")
        print("12. 将 [右摇杆] 向右推到底")
        print("13. 将 [右摇杆] 向上推到底")
        print("14. 按下 [DPAD-UP] (十字键上)")
        print("完成后，按 Ctrl+C 停止，并将所有输出复制给我。")
        print("--------------------------------------------------\n")

        last_report = None
        
        while True:
            report = device.read(64)
            if report:
                # 为了避免刷屏，只有当报文内容发生变化时才打印
                if report != last_report:
                    print_report(report)
                    last_report = report
            
            time.sleep(0.01)

    except OSError as e:
        print(f"\n错误: 无法打开设备。请检查设备是否连接，或尝试以管理员/root权限运行。")
        print(f"详细信息: {e}")
    except KeyboardInterrupt:
        print("\n\n分析结束。请复制上面的输出。")
    except Exception as e:
        print(f"\n发生未知错误: {e}")
    finally:
        if device:
            device.close()

正在尝试打开设备: VID=045e, PID=0b12
设备连接成功!
  制造商: Microsoft
  产品:   Controller

--- HID 报文分析器已启动 ---
请按照以下顺序，每次只做一个动作，并保持几秒钟：
1.  保持手柄完全静止 (非常重要！)
2.  只按住 [A] 键
3.  只按住 [B] 键
4.  只按住 [X] 键
5.  只按住 [Y] 键
6.  只按住 [LB] (左肩键)
7.  只按住 [RB] (右肩键)
8.  将 [LT] (左扳机) 压到底
9.  将 [RT] (右扳机) 压到底
10. 将 [左摇杆] 向右推到底
11. 将 [左摇杆] 向上推到底
12. 将 [右摇杆] 向右推到底
13. 将 [右摇杆] 向上推到底
14. 按下 [DPAD-UP] (十字键上)
完成后，按 Ctrl+C 停止，并将所有输出复制给我。
--------------------------------------------------

--------------------------------------------------
收到 19 字节报文:
[00-07] 20 ( 32)  00 (  0)  35 ( 53)  2C ( 44)  10 ( 16)  00 (  0)  00 (  0)  00 (  0)
[08-15] 00 (  0)  00 (  0)  42 ( 66)  FD (253)  CF (207)  FC (252)  87 (135)  FC (252)
[16-18] 5D ( 93)  FC (252)  00 (  0)

--------------------------------------------------
收到 19 字节报文:
[00-07] 20 ( 32)  00 (  0)  36 ( 54)  2C ( 44)  00 (  0)  00 (  0)  00 (  0)  00 (  0)
[08-15] 00 (  0)  00 (  0)  42 ( 66)  FD (253)  CF (207)  FC (252)  87 (135)  FC (252)
[16-18] 5D ( 93)  FC (252)  00 (  0

In [3]:
import hid
import struct
import time
import pyautogui

# --- PyAutoGUI 优化设置 ---
pyautogui.MINIMUM_DURATION = 0
pyautogui.MINIMUM_SLEEP = 0
pyautogui.PAUSE = 0
pyautogui.FAILSAFE = False

class XboxController:
    # --- [最终修正] 根据你的精确报文数据得出的按钮映射 ---
    # 你的控制器将按钮和DPAD放在了同一个字节 [4]
    # bit 0-3 是 DPAD, bit 4-7 是 A,B,X,Y
    BUTTON_MAP = {
        0: "DPAD_UP",
        1: "DPAD_DOWN",
        2: "DPAD_LEFT",
        3: "DPAD_RIGHT",
        4: "A",
        5: "B",
        6: "X",
        7: "Y",
    }
    # 肩键等在另一个字节 [5]
    BUTTON_MAP_2 = {
        0: "LB",
        1: "RB",
        2: "GUIDE", # 通常
        3: "SHARE", # 通常
        4: "MENU",
        5: "VIEW",
        6: "LS", # 左摇杆按下
        7: "RS", # 右摇杆按下
    }

    def __init__(self, vendor_id=0x045E, product_id=0x0B12):
        self.device = None
        try:
            self.device = hid.device()
            self.device.open(vendor_id, product_id)
            self.device.set_nonblocking(True)
            print("Connected:", self.device.get_manufacturer_string(),
                  self.device.get_product_string())
        except OSError as e:
            print(f"Error opening device: {e}")
            self.device = None

    def close(self):
        if self.device:
            self.device.close()

    def read(self):
        if not self.device:
            return None
            
        data = self.device.read(64)
        if not data or len(data) < 18:
            return None

        raw = bytes(data)

        # ----- [关键] 根据你的报文进行的精确解析 -----
        
        # 按钮 (DPAD, A, B, X, Y) 在字节 [4]
        buttons1_raw = raw[4]
        
        # 其他按钮 (LB, RB, Menu, View, etc.) 在字节 [5]
        buttons2_raw = raw[5]

        # 扳机键 LT/RT (10-bit unsigned) 在字节 [6]-[9]
        lt, rt = struct.unpack_from("<HH", raw, 6)

        # 摇杆 (16-bit signed) 在字节 [10]-[17]
        lx, ly, rx, ry = struct.unpack_from("<hhhh", raw, 10)
        
        # Y 轴向上为正，符合直觉，但 pyautogui 向下为正，所以我们在主程序中处理
        # 你的报文中，向上推摇杆已经是正值，所以这里不需要反转
        # ly, ry = -ly, -ry 

        # 合并所有按钮状态
        buttons = self._decode_buttons(buttons1_raw, self.BUTTON_MAP)
        buttons.update(self._decode_buttons(buttons2_raw, self.BUTTON_MAP_2))

        return {
            "buttons": buttons,
            "lt_norm": lt / 1023.0,
            "rt_norm": rt / 1023.0,
            "lx_norm": self._normalize_axis(lx),
            "ly_norm": self._normalize_axis(ly),
            "rx_norm": self._normalize_axis(rx),
            "ry_norm": self._normalize_axis(ry),
        }

    def _decode_buttons(self, bitmask, button_map):
        return {name: bool(bitmask & (1 << bit)) for bit, name in button_map.items()}

    def _normalize_axis(self, v):
        v = int(v)
        if v < 0: return max(-1.0, v / 32768.0)
        else: return min(1.0, v / 32767.0)


if __name__ == "__main__":
    MOUSE_SENSITIVITY = 25 
    DEADZONE = 0.15

    try:
        xbox = XboxController() 
        if not xbox.device:
            raise OSError("Controller not found or could not be opened.")

        print("\nController decoded successfully! Mouse control is active.")
        print("A = Left Click | B = Right Click | Ctrl+C to exit.")
        print("-" * 50)

        a_button_pressed = False
        b_button_pressed = False
        
        while True:
            state = xbox.read()
            if state:
                # --- 鼠标移动 ---
                lx = state['lx_norm']
                ly = state['ly_norm']
                
                if abs(lx) < DEADZONE: lx = 0
                if abs(ly) < DEADZONE: ly = 0
                
                if lx != 0 or ly != 0:
                    x_move = (lx ** 3) * MOUSE_SENSITIVITY
                    # 你的报文向上推摇杆是正值，而pyautogui向下是正值，所以需要一个负号
                    y_move = -(ly ** 3) * MOUSE_SENSITIVITY
                    pyautogui.move(x_move, y_move)
                
                # --- 鼠标点击 ---
                if state['buttons'].get('A') and not a_button_pressed:
                    pyautogui.mouseDown(button='left')
                    a_button_pressed = True
                elif not state['buttons'].get('A') and a_button_pressed:
                    pyautogui.mouseUp(button='left')
                    a_button_pressed = False
                    
                if state['buttons'].get('B') and not b_button_pressed:
                    pyautogui.mouseDown(button='right')
                    b_button_pressed = True
                elif not state['buttons'].get('B') and b_button_pressed:
                    pyautogui.mouseUp(button='right')
                    b_button_pressed = False

                # --- 调试输出 ---
                pressed_buttons = sorted([name for name, pressed in state["buttons"].items() if pressed])
                print(
                    f"Stick:({state['lx_norm']:.2f}, {state['ly_norm']:.2f}) "
                    f"LT:{state['lt_norm']:.2f} RT:{state['rt_norm']:.2f} "
                    f"Buttons: {pressed_buttons}      ", end='\r'
                )

            time.sleep(0.001) # 可以尝试更小的延迟

    except OSError as e:
        print(f"\nError: {e}")
    except KeyboardInterrupt:
        print("\nExiting.")
    finally:
        if 'xbox' in locals() and xbox:
            xbox.close()

Connected: Microsoft Controller

Controller decoded successfully! Mouse control is active.
A = Left Click | B = Right Click | Ctrl+C to exit.
--------------------------------------------------
Stick:(-0.00, -0.01) LT:0.00 RT:0.00 Buttons: []            ']      
Exiting.
